In [1]:
python3 main.py

SyntaxError: invalid syntax (293366439.py, line 1)

In [2]:
%run main.py

INFO:scene_setup:Starting scene setup with ray tracing...


XLA can lead to reduced numerical precision. Use with care.


DEBUG:scene_setup:Debugging Base station state:
DEBUG:scene_setup:- Position: [10.  10.   4.5]
DEBUG:scene_setup:- Orientation: [  0.   0. -90.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000001CB22DE65D0>
DEBUG:scene_setup:Debugging RIS state:
DEBUG:scene_setup:- Position: [10.  19.5  2.5]
DEBUG:scene_setup:- Orientation: [  0. -90.   0.]
DEBUG:scene_setup:Debugging AGV_0 state:
DEBUG:scene_setup:- Position: [12.   5.   0.5]
DEBUG:scene_setup:- Orientation: [0. 0. 0.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000001CB22E0A390>
DEBUG:scene_setup:Debugging AGV_1 state:
DEBUG:scene_setup:- Position: [ 8.  15.   0.5]
DEBUG:scene_setup:- Orientation: [0. 0. 0.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000001CB22DF7B90>
INFO:scene_setup:Scene setup completed successfully
INFO:scene_setup:- Transmitters: 7
INFO:scene_setup:- Receivers: 2
INFO:scene_setup:- RIS: 1
INFO:sc

Generating channel data...
Generating channel data...
Channel data generation completed
Channel data saved to: C:\Users\snatanzi\Desktop\Causal-XAI\src\results\channel_data.h5

Channel Data Statistics:
Channel matrices shape: (1, 2, 1, 7, 128, 1, 1024)
Path delays shape: (1, 2, 7, 2)
Number of receivers: 2
Execution completed


In [6]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from config import SmartFactoryConfig
from channel_generator import SmartFactoryChannel



DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [7]:
# Initialize configuration
config = SmartFactoryConfig()

# Create channel generator
channel_gen = SmartFactoryChannel(config)

# Generate channel
channel_data = channel_gen.generate_channel()

INFO:scene_setup:Starting scene setup with ray tracing...
DEBUG:scene_setup:Debugging Base station state:


XLA can lead to reduced numerical precision. Use with care.


DEBUG:scene_setup:- Position: [10.  10.   4.5]
DEBUG:scene_setup:- Orientation: [  0.   0. -90.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000001CBA3C29050>
DEBUG:scene_setup:Debugging RIS state:
DEBUG:scene_setup:- Position: [10.  19.5  2.5]
DEBUG:scene_setup:- Orientation: [  0. -90.   0.]
DEBUG:scene_setup:Debugging AGV_0 state:
DEBUG:scene_setup:- Position: [12.   5.   0.5]
DEBUG:scene_setup:- Orientation: [0. 0. 0.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000001CB9CAB1390>
DEBUG:scene_setup:Debugging AGV_1 state:
DEBUG:scene_setup:- Position: [ 8.  15.   0.5]
DEBUG:scene_setup:- Orientation: [0. 0. 0.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000001CBA3BEAB10>
INFO:scene_setup:Scene setup completed successfully
INFO:scene_setup:- Transmitters: 7
INFO:scene_setup:- Receivers: 2
INFO:scene_setup:- RIS: 1
INFO:scene_setup:- Objects: 0
INFO:scene_setup:- Ray tr

RuntimeError: Error generating channel response: Unknown method 'image'